In [1]:
import pandas as pd
import numpy as np
import os
import multiprocessing
import time
import Load_Data_Patient
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
from dateutil import parser
from datetime import datetime, time
import itertools
from tqdm import tqdm
import sys
import warnings
import json
import datetime
import Calculate_Time_Domain as td
import time
import dask.dataframe as dd
from dask.multiprocessing import get
import dask
if not sys.warnoptions:
    warnings.simplefilter("ignore")
# pbar = tqdm(total=36)
# pbar.update(1)
# pbar.close()

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
def Calculate_File_Pure(File, Interval, End):
    global pbar
    try:
        pbar.update(1)
        File = "Patient_Data_Files//" + str(File)
        Segmented = Load_Data_Patient.Get_Segmented_Table(File, Interval, End)

        Test_Table = Segmented[["RR Intervals", "Symbols"]]
        Test_Table.reset_index(drop=True, inplace=True)
        Test_Table["Mask"] = 1
        Test_Table["Reset"] = 0
        Test_Table["Percentage_Difference"] = 0

        Allowed_Symbols = ["N", "F", "+,(N", "+,(VT"]

        Test_Table["Index"] = Test_Table.index.values.tolist()
        Allowed_Symbols = ["N", "F"]

        def Symbol_Checker_Mask(Index):

            Current = Test_Table.loc[Index, "Symbols"]

            if Current in Allowed_Symbols:
                Test_Table.loc[Index,"Mask"] = 1
            else:
                Test_Table.loc[Index,"Mask"] = 0
                try:
                    if Test_Table.loc[Index+1,"Symbols"] in Allowed_Symbols:
                        Test_Table.loc[Index+2,"Reset"] = 1
                    else:
                        pass 
                except:
                    pass

        Test_Table["Index"].apply(Symbol_Checker_Mask)

        def Percentage_Generator_Column(Index):
            global Base
            try:
                To_Compare = Test_Table.loc[Index,"RR Intervals"]
                Percentage_Difference = abs(((Base-To_Compare)/Base)*100)
                Test_Table.loc[Index,"Percentage_Difference"] = Percentage_Difference
                Base = Test_Table.loc[Index,"RR Intervals"]
            except:
                pass

        Base = Test_Table["RR Intervals"][1]
        Test_Table["Index"].apply(Percentage_Generator_Column)

        def Previous_Symbol_Checker(Index):

            Current_Symbols = Test_Table.loc[Index]["Symbols"]

            try:
                Previous_Symbols = Test_Table.loc[Index-1]["Symbols"]
                if Previous_Symbols in Allowed_Symbols:
                    pass

                elif Current_Symbols=="N" and Previous_Symbols=="V":
                    Test_Table.loc[Index,"Mask"] = 1

                else:
                    Test_Table.loc[Index,"Mask"] = 0
            except:
                pass


        Test_Table["Index"].apply(Previous_Symbol_Checker)

        def Isolated_Elemets_Percentage(Index):

            try:
                Previous = Test_Table.loc[Index-1, "Percentage_Difference"]
                Current = Test_Table.loc[Index, "Percentage_Difference"]
                Future = Test_Table.loc[Index+1, "Percentage_Difference"]

                if Previous >= 15 and Future >= 15:
                    if Current <=15:
                        Test_Table.loc[Index, "Mask"] = 0
                    else:
                        pass
                elif Previous <= 15 and Future <= 15:
                    if Current >=15:
                        Test_Table.loc[Index, "Mask"] = 0
                    else:
                        pass
            except:
                pass

        Test_Table["Index"].apply(Isolated_Elemets_Percentage)

        Indexes = np.array(Test_Table.loc[Test_Table["Symbols"] == "+,(SIGNALLOST"].index)

        for i in range(Indexes.shape[0]):
            try:

                Current = int(Indexes[i])
                Future = int(Indexes[i+1])
                Sum = np.sum(Test_Table.loc[range(Current, Future), "RR Intervals"])
                if Sum <= 2:
                    Test_Table.drop(range(Current,Future), inplace=True)
                    del Current
                    del Future
                    del Sum
                else:
                    del Current
                    del Future
                    del Sum
            except:
                pass

        Test_Table.reset_index(drop=True, inplace=True)
        Test_Table["Index"] = Test_Table.index.values.tolist()

        def Signal_Lost_Elements(Index):

            try:
                if Test_Table.loc[Index, "Symbols"] == "+,(SIGNALLOST":
                    Test_Table.loc[Index+2, "Mask"] = 0
                else:
                    pass
            except:
                pass

        Test_Table["Index"].apply(Signal_Lost_Elements)
        Test_Table["Pure"] = 0

        def Isolated_Elemets_Mask(Index):

            try:
                Previous = Test_Table.loc[Index-1, "Mask"]
                Current = Test_Table.loc[Index, "Mask"]
                Future = Test_Table.loc[Index+1, "Mask"]

                if Previous == 0 and Future == 0:
                    Test_Table.loc[Index, "Mask"] = 0
            except:
                pass

        Test_Table["Index"].apply(Isolated_Elemets_Mask)

        Pure = Load_Data_Patient.Get_Pure_NNs(Test_Table)
        Pure.dropna(inplace=True)

        def Extremes(Index):

            if Pure.loc[Index, "RR Intervals"] < 0.42:
                Pure.loc[Index, "Pure"] = 0
            elif Pure.loc[Index, "RR Intervals"] > 1.2:
                Pure.loc[Index, "Pure"] = 0
            else:
                pass

        Pure["Index"].apply(Extremes)

        Pure["Percentage"] = 0
        Base = Pure.loc[Pure["Pure"] == 1].index[0]

        def Percentage_Generator_Column(Index):
            global Base
            if Pure.loc[Index, "Pure"] == 1:
                try:
                    To_Compare = Pure.loc[Index,"RR Intervals"]
                    Percentage_Difference = int(round(abs(((Base-To_Compare)/Base)*100)))
                    Pure.loc[Index,"Percentage"] = Percentage_Difference
                    Base = Pure.loc[Index,"RR Intervals"]
                except:
                    pass


        Pure["Index"].apply(Percentage_Generator_Column)

        Pure["Peglano"] = 0 
        def Peglanje(Index):
            if Pure.loc[Index, "Pure"] == 1:
                try:
                    if Pure.loc[Index, "Percentage"] > 5:
                        PreviousAVG = np.average([Pure.loc[Index-2, "RR Intervals"], Pure.loc[Index-1, "RR Intervals"]])
                        CurrentAVG = np.average([Pure.loc[Index, "RR Intervals"], Pure.loc[Index+1, "RR Intervals"]])
                        FutureAVG = np.average([Pure.loc[Index+2, "RR Intervals"], Pure.loc[Index+3, "RR Intervals"]])

                        Percentage_Difference_Previous = int(round(abs(((CurrentAVG-PreviousAVG)/PreviousAVG)*100)))
                        Percentage_Difference_Future = int(round(abs(((CurrentAVG-FutureAVG)/FutureAVG)*100)))

                        if Percentage_Difference_Previous <= 2 and Percentage_Difference_Future <= 2:
        #                     Peglano.append(Pure.loc[Index, "RR Intervals"])
                            Pure.loc[Index, "Peglano"] = 1
                        else:
                            pass
                    else:
        #                 Peglano.append(Pure.loc[Index, "RR Intervals"])
                        Pure.loc[Index, "Peglano"] = 1
                except:
                    pass
            else:
                pass

        Peglano = []
        Pure["Index"].apply(Peglanje)

        def Isolated_Elemets_Mask(Index):

            try:
                Previous = Test_Table.loc[Index-1, "Peglano"]
                Current = Test_Table.loc[Index, "Peglano"]
                Future = Test_Table.loc[Index+1, "Peglano"]

                if Previous == 0 and Future == 0:
                    Test_Table.loc[Index, "Peglano"] = 0
            except:
                pass

        Pure["Index"].apply(Isolated_Elemets_Mask)

        Peglano = np.array(Pure.loc[Pure["Peglano"] == 1]["RR Intervals"])

        Results = Load_Data_Patient.Calculate_Pure_Td(Peglano)

        f= open("5h_Interval_Results//" + str(Interval) + "h_"+ str(End) +"h_AllPatientsParralel.txt","a+")
        To_Write = str(File).split("//")[-2] + "|" + str(Interval) + "|" + str(End) + "|" + str(File).split("//")[-1] + "|" + str(Results["Pure_SDNN"]) + "|" + str(Results["Pure_ASDNN"]) + "|" + str(Results["Pure_SDANN"]) + "|" + str(Results["Pure_NN50"]) + "|" + str(Results["Pure_pNN50"]) + "|" + str(Results["Pure_rMSSD"]) + "\n"
        f.write(To_Write)
        f.close()
        del Results
    except:
        pbar.update(1)

In [5]:
All_Files_Table = pd.read_excel("../Make_Input_Table/Valids.xlsx")

In [6]:
All_Files_Table["Baseline"].unique()

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24,  1], dtype=int64)

In [7]:
Five = All_Files_Table.loc[All_Files_Table["Before"] == 5]
Five.head()

,Patient_ID,File,Before,Baseline,Valid,Index
22,C0161,C0161//22_11_2018_000000-22_11_2018_092830.ann...,5,6,True,22
29,C0161,C0161//22_11_2018_000000-22_11_2018_092830.ann...,5,7,True,29
37,C0161,C0161//22_11_2018_000000-22_11_2018_092830.ann...,5,8,True,37
82,C0161,C0161//20_11_2018_120700-20_11_2018_192830.ann...,5,17,True,82
89,C0161,C0161//20_11_2018_120700-20_11_2018_192830.ann...,5,18,True,89


In [8]:
Five.shape[0]

3842

In [9]:
ddata = dd.from_pandas(Five, npartitions=30)

In [ ]:
client = Client()

In [ ]:
client

In [ ]:
Failed = []
Apply = ddata.apply(lambda X: Calculate_File_Pure(X['File'],
                                                  X["Before"],
                                                  X["Baseline"]),
                                                  axis=1,
                                                  meta=(object))
# 10 patitions That took 537.7677550315857 seconds 1@14 150
# 30 partitiona That took 482.91249990463257 seconds 1@5 250 / 121p 501s 1@5 250r
# 100 partiotions That took 488.9536120891571 seconds 1@6 184
# That took 9420.142078876495 seconds
# That took 16390.28028845787 seconds
starttime = time.time()
# with ProgressBar():
Apply.compute()
print('That took {} seconds'.format(time.time() - starttime))
# pbar.close()

4283it [27:25:40, 51.13s/it] 